In [41]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from pymongo import MongoClient

import pprint

import pandas as pd

# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup

import json


In [42]:
client = MongoClient('localhost', 27017)
racing = client['results']
source = racing['source code']
runners = racing['indiv']

In [43]:
def parse_results(data, distance, ag):
    soup = BeautifulSoup(data, 'html.parser')
    for i, result in enumerate(soup.findAll("div",'rms-grid-item')):
        t = result.findAll("span", "long-text")[0].text[4:]
        a = int(result.findAll("span", "ng-binding ng-scope")[0].text[1:])
        n = result.findAll('div', 'name rms-grid-line ng-binding')[0].text.strip()
        runners.insert({'name': n, 'time':t, 'age': a, 'ag': ag, 'dist': distance})

In [44]:
urls = [['https://results.nyrr.org/event/M2019/finishers#g=M&page=1&af=40&at=44', 'Men 40-44', 'Mar'], ['https://results.nyrr.org/event/M2019/finishers#g=M&page=1&af=45&at=49', 'Men 45-49', 'Mar'], ['https://results.nyrr.org/event/M2019/finishers#g=F&page=1&af=25&at=29', 'Women 25-29', 'Mar'], ['https://results.nyrr.org/event/M2019/finishers#g=F&page=1&af=30&at=34', 'Women 30-34', 'Mar'], ['https://results.nyrr.org/event/5AV-19/finishers#g=F&page=1&af=25&at=29', 'Women 25-29', 'Mile'], ['https://results.nyrr.org/event/5AV-19/finishers#g=F&page=1&af=30&at=34', 'Women 30-34', 'Mile'],['https://results.nyrr.org/event/5AV-19/finishers#g=M&page=1&af=40&at=44', 'Men 40-44', 'Mile'], ['https://results.nyrr.org/event/5AV-19/finishers#g=M&page=1&af=45&at=49', 'Men 45-49', 'Mile']]
test_urls = [['https://results.nyrr.org/event/20JK10/finishers#g=M&page=1&af=20&at=24', 'Men 20-24', '10k'], ['https://results.nyrr.org/event/20JK10/finishers#g=F&page=1&af=20&at=24', 'Women 20-24', '10k']]
## get results
def git_er_done(urls_list):
    for i in range(len(urls_list)):
        source_code = scrape(urls_list[i][0])
        print(urls_list[i][0])
#         source_code = driver.page_source
        racing.source.insert({'name': urls_list[i][1], 'source': source_code})
        parse_results(source_code, urls_list[i][2], urls_list[i][1])

In [45]:
def scrape(url):
    chrome_driver_path = "/Users/annebarry/galvanize/running_rivalries/chromedriver"
    driver = webdriver.Chrome(executable_path = chrome_driver_path)
    driver.get(url)
    print('made it to the scrape', url)
    time.sleep(10)
    button = driver.find_elements_by_class_name('button-load-more')
    while True:
        try:
            button[1].click()
            time.sleep(12)
        except:
            break
    source = driver.page_source
    return source

In [ ]:
git_er_done(urls)

made it to the scrape https://results.nyrr.org/event/M2019/finishers#g=M&page=1&af=40&at=44
https://results.nyrr.org/event/M2019/finishers#g=M&page=1&af=40&at=44


/Users/annebarry/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  if __name__ == '__main__':
/Users/annebarry/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  import sys


made it to the scrape https://results.nyrr.org/event/M2019/finishers#g=M&page=1&af=45&at=49
https://results.nyrr.org/event/M2019/finishers#g=M&page=1&af=45&at=49
made it to the scrape https://results.nyrr.org/event/M2019/finishers#g=F&page=1&af=25&at=29
https://results.nyrr.org/event/M2019/finishers#g=F&page=1&af=25&at=29
made it to the scrape https://results.nyrr.org/event/M2019/finishers#g=F&page=1&af=30&at=34


In [ ]:
individuals = pd.DataFrame(list(runners_test.find()))

In [ ]:
print(individuals)

In [ ]:
def parse_results(data, distance, ag):
    soup = BeautifulSoup(data, 'html.parser')
    for i, result in enumerate(soup.findAll("div",'rms-grid-item')):
        t = result.findAll("span", "long-text")[0].text[4:]
        a = int(result.findAll("span", "ng-binding ng-scope")[0].text[1:])
        n = result.findAll('div', 'name rms-grid-line ng-binding')[0].text.strip()
        runners.insert({'name': n, 'time':t, 'age': a, 'ag': ag, 'dist': distance})

In [40]:
list(test_runners.find())

[{'_id': ObjectId('5f16fc5fc94e506c54091f4f'),
  'name': 'Teshome Mekonen',
  'time': '0:29:23',
  'age': 24,
  'ag': 'Men 20-24',
  'dist': '10k'},
 {'_id': ObjectId('5f16fc5fc94e506c54091f50'),
  'name': 'Abdulmenan Kasim Gelato',
  'time': '0:31:18',
  'age': 22,
  'ag': 'Men 20-24',
  'dist': '10k'},
 {'_id': ObjectId('5f16fc5fc94e506c54091f51'),
  'name': 'Brandon Shirazi',
  'time': '0:34:39',
  'age': 22,
  'ag': 'Men 20-24',
  'dist': '10k'},
 {'_id': ObjectId('5f16fc5fc94e506c54091f52'),
  'name': 'Benjamin Barton',
  'time': '0:37:13',
  'age': 23,
  'ag': 'Men 20-24',
  'dist': '10k'},
 {'_id': ObjectId('5f16fc5fc94e506c54091f53'),
  'name': 'Michael Gabrellian',
  'time': '0:37:44',
  'age': 24,
  'ag': 'Men 20-24',
  'dist': '10k'},
 {'_id': ObjectId('5f16fc5fc94e506c54091f54'),
  'name': 'Arno Rupp',
  'time': '0:38:06',
  'age': 24,
  'ag': 'Men 20-24',
  'dist': '10k'},
 {'_id': ObjectId('5f16fc5fc94e506c54091f55'),
  'name': 'Nolan Rooney',
  'time': '0:39:03',
  'age'

In [ ]:
individuals = pd.DataFrame(list(individual_results.find()))

In [ ]:


driver.get('https://results.nyrr.org/event/M2019/finishers#g=M&page=1&af=40&at=44')
button = driver.find_elements_by_class_name('button-load-more')
time.sleep(20)

while True:
    try:
        button[1].click()
        time.sleep(random.randint(15)) 
    except:
        break
        
m_40_44 = driver.page_source

In [ ]:
m_40_44